Attribute Information:

The data set contains raw process sensor data (i.e. without feature extraction) which are structured as matrices (tab-delimited) with the rows representing the cycles and the columns the data points within a cycle. The sensors involved are:

Sensor		Physical quantity		Unit		Sampling rate
PS1		Pressure			bar		100 Hz
PS2		Pressure			bar		100 Hz
PS3		Pressure			bar		100 Hz
PS4		Pressure			bar		100 Hz
PS5		Pressure			bar		100 Hz
PS6		Pressure			bar		100 Hz
EPS1		Motor power			W		100 Hz
FS1		Volume flow			l/min		10 Hz
FS2		Volume flow			l/min		10 Hz
TS1		Temperature			Â°C		1 Hz
TS2		Temperature			Â°C		1 Hz
TS3		Temperature			Â°C		1 Hz
TS4		Temperature			Â°C		1 Hz
VS1		Vibration			mm/s		1 Hz
CE		Cooling efficiency (virtual)	%		1 Hz
CP		Cooling power (virtual)		kW		1 Hz
SE		Efficiency factor		%		1 Hz



Has Missing Values?

No

In [104]:
import pandas as pd
import numpy as np
from numpy import loadtxt
import matplotlib.pyplot as plt

#PS: Pressure, bar, 100 Hz --> 100 Messungen pro Skeunde
#EPS: Motor power, W, 100 Hz
#FS: Volume flow, l/min, 10 Hz --> 10 Messungen pro Sekunde
#TS: Temperature, Celsius, 1 Hz --> 1 Messung pro Sekunde
#VS: Vibration, mm/s, 1Hz
#CE: Cooling efficiency (virtual), %, 1 Hz
#CP: Cooling power (virtual), kW, 1 Hz
#SE: Efficency factor, %, 1 Hz


input_path = "input_data/"
file_names = [
    "ce", "cp", "eps1", "se", "vs1", 
    "fs1", "fs2", 
    "ps1", "ps2", "ps3", "ps4", "ps5", "ps6",
    "ts1", "ts2", "ts3", "ts4", "target"
]

data = {}

for file_name in file_names:
    data[file_name] = loadtxt(f"{input_path}{file_name}.txt")

In [105]:
for file_name in file_names:
    print(f"Shape of {file_name}:{data[file_name].shape}")

Shape of ce:(2205, 60)
Shape of cp:(2205, 60)
Shape of eps1:(2205, 6000)
Shape of se:(2205, 60)
Shape of vs1:(2205, 60)
Shape of fs1:(2205, 600)
Shape of fs2:(2205, 600)
Shape of ps1:(2205, 6000)
Shape of ps2:(2205, 6000)
Shape of ps3:(2205, 6000)
Shape of ps4:(2205, 6000)
Shape of ps5:(2205, 6000)
Shape of ps6:(2205, 6000)
Shape of ts1:(2205, 60)
Shape of ts2:(2205, 60)
Shape of ts3:(2205, 60)
Shape of ts4:(2205, 60)
Shape of target:(2205, 5)


In [106]:
try:
    target_columns = ['Cooler_Condition', 'Valve_Condition', 
                      'Internal_Pump_Leakage', 'Hydraulic_Accumulator', 
                      'Stable_Flag']
    df_target = pd.DataFrame(data['target'], columns=target_columns)
    df_target = df_target['Valve_Condition']  # Nur 'Valve_Condition' verwenden
    del data['target']  # 'target' aus den übrigen Daten entfernen
except Exception as e:
    print(f"Fehler beim Verarbeiten von 'target': {e}")

#df_target.head()

In [107]:
df_target.describe()

count    2205.000000
mean       90.693878
std        10.681802
min        73.000000
25%        80.000000
50%       100.000000
75%       100.000000
max       100.000000
Name: Valve_Condition, dtype: float64

In [ ]:
fig, axes = plt.subplots(6, 3, figsize=(15, 20))
axes = axes.flatten()  


for i, key in enumerate(data):
    ax = axes[i]
    ax.plot(data[key], color = 'blue', linewidth = 1)
    ax.plot(np.mean(data[key], axis=1), color = 'red') 
    ax.set_title(key) 

plt.tight_layout()
plt.show()

In [ ]:
for key, array in data.items():
    print(f"{key}: Mean = {round((np.mean(array)),4)}, Std = {round((np.std(array)),4)}, Min = {np.min(array)}, Max = {np.max(array)}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

for key in data: 
    # Erstelle ein 2D-Array (zum Beispiel ein einfaches Bild oder eine Matrix)
    image = data[key]

    # Berechne die 2D-Fourier-Transformation
    f_transform = np.fft.fft2(image)

    # Verschiebe den Nullfrequenzanteil in die Mitte des Spektrums
    f_transform_shifted = np.fft.fftshift(f_transform)

    # Berechne das Magnitudespektrum (Betrag)
    magnitude_spectrum = np.abs(f_transform_shifted)

    # Berechne das Phasenspektrum (Phase)
    phase_spectrum = np.angle(f_transform_shifted)

    # Summiere das Magnitudespektrum entlang der Spalten (horizontale Richtung)
    magnitude_spectrum_sum_cols = np.sum(magnitude_spectrum, axis=0)

    # Plot des Magnitudespektrums als Summe entlang der Spalten
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(magnitude_spectrum_sum_cols)
    plt.title(f"{key}: Magnitude Spectrum (Summe entlang der Spalten)")
    plt.xlabel("Spalten-Index")
    plt.ylabel("Summierte Magnitude")

    # Plot des Phasenspektrums
    plt.subplot(1, 2, 2)
    plt.imshow(phase_spectrum, cmap='twilight', aspect='auto')
    plt.title(f"{key}: Phase Spectrum")
    plt.colorbar(label='Phase (radians)')
    plt.tight_layout()
    plt.show()


In [115]:
len(data['ce'].mean(axis = 1))

2205

In [116]:
import numpy as np

# Funktion zur Feature-Extraktion
def extract_features(data):
    # Sicherstellen, dass die Eingabe ein 2D-Array ist
    features = {}
    # Mittelwert, Median, Standardabweichung, Minimum, Maximum und Spannweite spaltenweise
    features['mean'] = np.mean(data, axis=0)  # Mittelwert über Spalten
    features['median'] = np.median(data, axis=0)  # Median über Spalten
    features['std'] = np.std(data, axis=0)  # Standardabweichung über Spalten
    features['min'] = np.min(data, axis=0)  # Minimum über Spalten
    features['max'] = np.max(data, axis=0)  # Maximum über Spalten
    features['range'] = features['max'] - features['min']  # Spannweite über Spalten

    # Differenzen entlang der Zeilen
    diffs = np.diff(data, axis=0)  # Differenzen entlang der Zeilen
    features['mean_diff'] = np.mean(diffs, axis=0)  # Mittelwert der Differenzen
    features['std_diff'] = np.std(diffs, axis=0)  # Standardabweichung der Differenzen

    return features


In [127]:
means = []
for key in data:
    means.append(data[key].mean(axis = 1))

In [132]:
ps1_mean = data['ps1'].mean(axis=1)
df = pd.DataFrame(ps1_mean, columns=["test"])
df

,test
0,160.673492
1,160.603320
2,160.347720
3,160.188088
4,160.000472
...,...
2200,161.227572
2201,161.206070
2202,161.192120
2203,161.208917


In [136]:
# Transponiere die Daten, damit 17 Listen zu 17 Spalten werden
transposed_data = list(map(list, zip(*means)))

# Erstelle den DataFrame
df = pd.DataFrame(transposed_data, columns=[f"Spalte{i+1}" for i in range(17)])

# Ausgabe des DataFrames
print(df)
print(df.shape) 

        Spalte1   Spalte2      Spalte3    Spalte4   Spalte5   Spalte6  \
0     39.601350  1.862750  2538.929167  59.157183  0.576950  6.709815   
1     25.786433  1.255550  2531.498900  59.335617  0.565850  6.715315   
2     22.218233  1.113217  2519.928000  59.543150  0.576533  6.718522   
3     20.459817  1.062150  2511.541633  59.794900  0.569267  6.720565   
4     19.787017  1.070467  2503.449500  59.455267  0.577367  6.690308   
...         ...       ...          ...        ...       ...       ...   
2200  46.628517  2.160600  2543.911033  59.033100  0.550833  6.689930   
2201  46.689817  2.151450  2543.411333  59.068000  0.547483  6.692182   
2202  46.472300  2.143300  2542.729767  59.132350  0.545233  6.693277   
2203  46.544967  2.148483  2544.046333  58.970800  0.537017  6.684128   
2204  46.647933  2.157050  2543.818300  59.053900  0.546583  6.692302   

        Spalte7     Spalte8     Spalte9  Spalte10   Spalte11  Spalte12  \
0     10.304592  160.673492  109.466914  1.991475

In [133]:
columns = [
    "ce", "cp", "eps1", "se", "vs1", 
    "fs1", "fs2", 
    "ps1", "ps2", "ps3", "ps4", "ps5", "ps6",
    "ts1", "ts2", "ts3", "ts4"
]
df = pd.DataFrame(means, columns=columns)

ValueError: 17 columns passed, passed data had 2205 columns

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Extrahierte Features und Labels (ersetze dies mit deinem Datensatz)

# Features und Zielvariable
X = features_df
y = df_target

X.shape

y.shape

# Datenaufteilung
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# AdaBoost-Modell
ada_model = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1), n_estimators=50, random_state=42)

# Training
ada_model.fit(X_train, y_train)

# Vorhersagen
y_pred = ada_model.predict(X_test)

# Modellbewertung
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
